In [5]:
import numpy as np
import struct
import sys

import pdb


import scipy.misc

caffe_root = '/home/mjhuria/caffe/'  # this file is expected to be in {caffe_root}/examples
sys.path.insert(0, caffe_root + 'python')

import caffe

import matplotlib.pyplot as plt
%matplotlib inline







caffe.set_mode_cpu()
model_def = '/media/mjhuria/New Volume/Vikram/EC551/Final/27-04/Test.prototxt'
# model_def = '/home/mjhuria/caffe/to_train/caffe_alexnet_train_iter_10000.caffemodel'
model_weights =  '/media/mjhuria/New Volume/Vikram/EC551/Final/27-04/solver_iter_1785.caffemodel'
model_weights_new =  '/media/mjhuria/New Volume/Vikram/EC551/Final/27-04/solver_iter_50000.caffemodel'
net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perfor

new_net = caffe.Net(model_def,      # defines the structure of the model
                model_weights_new,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perfor
print [(k, v.data.shape) for k, v in net.blobs.items()]
print [(k, v[0].data.shape, v[0].data.min(), v[0].data.max()) for k, v in net.params.items()]

conv1_f = net.params['conv1'][0].data   # conv kernels
conv1_fb = net.params['conv1'][1].data  # conv bias
conv2_f = net.params['conv2'][0].data   # conv kernels
conv2_fb = net.params['conv2'][1].data  # conv bias
ip1_f = net.params['fc6'][0].data       # inner product weights
ip1_fb = net.params['fc6'][1].data      # inner product bias
ip2_f = net.params['fc8'][0].data       # inner product weights
ip2_fb = net.params['fc8'][1].data      # inner product bias
ip2_fb_new = new_net.params['fc8'][1].data      # inner product bias



[('data', (1, 3, 227, 227)), ('conv1', (1, 96, 112, 112)), ('norm1', (1, 96, 112, 112)), ('conv2', (1, 96, 54, 54)), ('norm2', (1, 96, 54, 54)), ('pool1', (1, 96, 27, 27)), ('conv3', (1, 256, 27, 27)), ('norm3', (1, 256, 27, 27)), ('pool2', (1, 256, 13, 13)), ('conv4', (1, 384, 13, 13)), ('conv5', (1, 384, 13, 13)), ('conv6', (1, 256, 13, 13)), ('pool6', (1, 256, 6, 6)), ('fc6', (1, 4096)), ('fc7', (1, 4096)), ('fc8', (1, 17)), ('prob', (1, 17))]
[('conv1', (96, 3, 5, 5), -0.06573832, 0.061630037), ('conv2', (96, 96, 5, 5), -0.04730388, 0.05208222), ('conv3', (256, 48, 5, 5), -0.044767905, 0.075381145), ('conv4', (384, 256, 3, 3), -0.04601701, 0.04862725), ('conv5', (384, 192, 3, 3), -0.04486917, 0.052669864), ('conv6', (256, 192, 3, 3), -0.057604782, 0.05344401), ('fc6', (4096, 9216), -0.026020113, 0.026968492), ('fc7', (4096, 4096), -0.02452982, 0.028030848), ('fc8', (17, 4096), -0.061515596, 0.06983797)]


In [6]:
print ip2_fb
print ip2_fb_new

[ 0.02085068 -0.01552874 -0.08721516  0.00019031  0.02808356  0.10867587
 -0.08544107 -0.04722133 -0.05846792  0.11105247 -0.01207688 -0.04492364
 -0.02381523  0.12786132  0.03540667  0.07877587 -0.13620824]
[-0.00584016  0.03025899 -0.10609204 -0.02615358 -0.02480794  0.11105154
 -0.11218703 -0.02903258 -0.0639886   0.12814619 -0.04865126 -0.01865094
 -0.00483673  0.13273832  0.01666117  0.15313369 -0.13174906]


In [4]:
# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
print({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
#net.blobs['data'].reshape(1,1,48,96)
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR
image = caffe.io.load_image('/media/mjhuria/New Volume/Vikram/EC551/Project/Neural_Nets/17flowers_adjusted/jpg/9/image_0161.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data', image)
#net.blobs['data'].data[...] = caffe.io.load_image(caffe_root + "pKlot_lmdb/Dataset/Full_data/Val/2335.png",False)
print({'data': net.blobs['data'].data.shape})

out = net.forward()
print("Predicted class is #{}.".format(out['prob'][0].argmax()))
print out['prob'][0]

{'data': (1, 3, 227, 227)}
{'data': (1, 3, 227, 227)}
Predicted class is #9.
[1.5697801e-01 1.1527365e-02 2.0401238e-03 3.5176173e-02 1.9070482e-03
 7.5618058e-02 2.8775432e-03 1.0467322e-02 7.5479940e-04 4.7263613e-01
 7.8118574e-05 2.9668570e-03 2.9059574e-02 7.0100479e-02 6.4162078e-04
 1.2710558e-01 6.5266897e-05]


In [16]:
####################################
## write float learning data to file
####################################
f = open("feat_pklot_conv1_f_f32.txt", "wb")
fb = open("feat_pklot_conv1_fb_f32.txt", "wb")
f.write(struct.pack('4i',*conv1_f.shape))
fb.write(struct.pack('1i',*conv1_fb.shape))
f.write(np.round(1).astype(np.int32))
fb.write(np.round(1).astype(np.int32))
conv1_f.tofile(f)
conv1_fb.tofile(fb)
f.close()
fb.close()

f = open("feat_pklot_conv2_f_f32.txt", "wb")
fb = open("feat_pklot_conv2_fb_f32.txt", "wb")
f.write(struct.pack('4i',*conv2_f.shape))
fb.write(struct.pack('1i',*conv2_fb.shape))
conv2_norm = 127 / max( max(abs(conv2_f.min()), abs(conv2_f.max())), max(abs(conv2_fb.min()), abs(conv2_fb.max())))
f.write(np.round(1).astype(np.int32))
fb.write(np.round(1).astype(np.int32))
conv2_f.tofile(f)
conv2_fb.tofile(fb)
f.close()
fb.close()

f = open("feat_pklot_ip1_f_f32.txt", "wb")
fb = open("feat_pklot_ip1_fb_f32.txt", "wb")
f.write(struct.pack('2i',*ip1_f.shape))
fb.write(struct.pack('1i',*ip1_fb.shape))
ip1_norm = 127 / max( max(abs(ip1_f.min()), abs(ip1_f.max())), max(abs(ip1_fb.min()), abs(ip1_fb.max())))
f.write(np.round(1).astype(np.int32))
fb.write(np.round(1).astype(np.int32))
ip1_f.tofile(f)
ip1_fb.tofile(fb)
f.close()
fb.close()

f = open("feat_pklot_ip2_f_f32.txt", "wb")
fb = open("feat_pklot_ip2_fb_f32.txt", "wb")
f.write(struct.pack('2i',*ip2_f.shape))
fb.write(struct.pack('1i',*ip2_fb.shape))
ip2_norm = 127 / max( max(abs(ip2_f.min()), abs(ip2_f.max())), max(abs(ip2_fb.min()), abs(ip2_fb.max())))
f.write(np.round(1).astype(np.int32))
fb.write(np.round(1).astype(np.int32))
ip2_f.tofile(f)
ip2_fb.tofile(fb)
f.close()
fb.close()


####################################
## convert learning data to signed 8bit int and write to file
####################################
f = open("feat_pklot_conv1_f_s8.txt", "wb")
fb = open("feat_pklot_conv1_fb_s8.txt", "wb")
f.write(struct.pack('4i',*conv1_f.shape))
fb.write(struct.pack('1i',*conv1_fb.shape))
conv1_norm = 127 / max( max(abs(conv1_f.min()), abs(conv1_f.max())), max(abs(conv1_fb.min()), abs(conv1_fb.max())))
f.write(np.round(conv1_norm).astype(np.int32))
fb.write(np.round(conv1_norm).astype(np.int32))
conv1_f = conv1_norm * conv1_f    
conv1_fb = conv1_norm * conv1_fb  
conv1_f = np.round(conv1_f).astype(np.int8)
conv1_fb = np.round(conv1_fb).astype(np.int8)
conv1_f.tofile(f)
conv1_fb.tofile(fb)
f.close()
fb.close()
# print conv1_f

f = open("feat_pklot_conv2_f_s8.txt", "wb")
fb = open("feat_pklot_conv2_fb_s8.txt", "wb")
f.write(struct.pack('4i',*conv2_f.shape))
fb.write(struct.pack('1i',*conv2_fb.shape))
conv2_norm = 127 / max( max(abs(conv2_f.min()), abs(conv2_f.max())), max(abs(conv2_fb.min()), abs(conv2_fb.max())))
f.write(np.round(conv2_norm).astype(np.int32))
fb.write(np.round(conv2_norm).astype(np.int32))
conv2_f = conv2_norm * conv2_f    
conv2_fb = conv2_norm * conv2_fb  
conv2_f = np.round(conv2_f).astype(np.int8)
conv2_fb = np.round(conv2_fb).astype(np.int8)
conv2_f.tofile(f)
conv2_fb.tofile(fb)
f.close()
fb.close()

f = open("feat_pklot_ip1_f_s8.txt", "wb")
fb = open("feat_pklot_ip1_fb_s8.txt", "wb")
f.write(struct.pack('2i',*ip1_f.shape))
fb.write(struct.pack('1i',*ip1_fb.shape))
ip1_norm = 127 / max( max(abs(ip1_f.min()), abs(ip1_f.max())), max(abs(ip1_fb.min()), abs(ip1_fb.max())))
f.write(np.round(ip1_norm).astype(np.int32))
fb.write(np.round(ip1_norm).astype(np.int32))
ip1_f = ip1_norm * ip1_f
ip1_fb = ip1_norm * ip1_fb
ip1_f = np.round(ip1_f).astype(np.int16)
ip1_fb = np.round(ip1_fb).astype(np.int16)
ip1_f.tofile(f)
ip1_fb.tofile(fb)
f.close()
fb.close()

f = open("feat_pklot_ip2_f_s8.txt", "wb")
fb = open("feat_pklot_ip2_fb_s8.txt", "wb")
f.write(struct.pack('2i',*ip2_f.shape))
fb.write(struct.pack('1i',*ip2_fb.shape))
ip2_norm = 127 / max( max(abs(ip2_f.min()), abs(ip2_f.max())), max(abs(ip2_fb.min()), abs(ip2_fb.max())))
f.write(np.round(ip2_norm).astype(np.int32))
fb.write(np.round(ip2_norm).astype(np.int32))
ip2_f = ip2_norm * ip2_f
ip2_fb = ip2_norm * ip2_fb
ip2_f = np.round(ip2_f).astype(np.int16)
ip2_fb = np.round(ip2_fb).astype(np.int16)
ip2_f.tofile(f)
ip2_fb.tofile(fb)
f.close()
fb.close()
print ip2_fb.shape
print ip2_f.shape

(17,)
(17, 4096)


In [10]:
A=open('feat_pklot_conv1_f_s8.txt',"r")
print A

<open file 'feat_pklot_conv1_f_s8.txt', mode 'r' at 0x7fe4f74d8ed0>


In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [5]:
caffe.set_mode_cpu()


# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
print({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]

{'data': (1, 1, 48, 96)}


In [12]:
#net.blobs['data'].reshape(1,1,48,96)
net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(caffe_root + "pKlot_lmdb/Dataset/Full_data/Val/3000.png")[:,:,0:1])
#net.blobs['data'].data[...] = caffe.io.load_image(caffe_root + "pKlot_lmdb/Dataset/Full_data/Val/2335.png",False)
print({'data': net.blobs['data'].data.shape})

out = net.forward()
print("Predicted class is #{}.".format(out['prob'].argmax()))
print ""
print [(k, v.data.shape, v.data.min(), v.data.max()) for k, v in net.blobs.items()]
print ""
print [(k, v[0].data.shape, v[0].data.min(), v[0].data.max()) for k, v in net.params.items()]
print ""
print [(k, v[1].data.shape, v[1].data.min(), v[1].data.max()) for k, v in net.params.items()]

{'data': (1, 1, 48, 96)}
Predicted class is #1.

[('data', (1, 1, 48, 96), 19.069338, 254.97754), ('conv1', (1, 15, 22, 46), -501.96121, 206.58829), ('relu1', (1, 15, 22, 46), 0.0, 206.58829), ('conv2', (1, 20, 9, 14), -115.98009, 128.43457), ('relu2', (1, 20, 9, 14), 0.0, 128.43457), ('ip1', (1, 20), -17.467922, 41.461407), ('relu3', (1, 20), 0.0, 41.461407), ('ip2', (1, 2), -8.19205, 14.696536), ('prob', (1, 2), 1.147132e-10, 1.0)]

[('conv1', (15, 1, 5, 5), -0.36110434, 0.3486467), ('conv2', (20, 15, 5, 5), -0.12282064, 0.14657567), ('ip1', (20, 2520), -0.06060081, 0.071337216), ('ip2', (2, 20), -0.43557408, 0.40337211)]

[('conv1', (15,), -0.0016508795, 0.00060002686), ('conv2', (20,), -0.0014067695, 0.0016344107), ('ip1', (20,), -0.10476051, 0.25708494), ('ip2', (2,), -0.77703452, 0.7770344)]
